# Introduction

This program draws a "heat map" of COVID cases based on the Covid-19 dataset. The dataset includes information about
case counts over time. <br>
It relies on a pre-made country map JSON to make the Folium overlay, and a "better_names" csv which I wrote so that the
code wouldn't get too cluttered.

To Do: <br>
Figure out why France and Argentina display as the wrong color.

# Main Program

## Init

Imports standard libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mp
import folium as fol
import pycountry as pc
from urllib.request import urlopen
from json import load

Loads data files on GitHub (for Colab)

In [ ]:
COVID_DATA_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/covid_19_data.csv"
BETTER_NAMES_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/better_names.csv"
COUNTRIES_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/countries.json"
MERGED_COUNTRIES_URL = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/merged_countries.json"

# df = pd.read_csv(COVID_DATA_URL)
# bn = pd.read_csv(BETTER_NAMES_URL)
# ct = load(urlopen(COUNTRIES_URL))
# mc = load(urlopen(MERGED_COUNTRIES_URL))

Loads Data Files (for execution locally/ GitHub).

In [ ]:
df = pd.read_csv("../Dat/covid_19_data.csv")
bn = pd.read_csv("../Dat/better_names.csv")
ct = load(open("../Dat/countries.json"))
mc = load(open("../Dat/merged_countries.json"))


## -- Run --

Function which changes the poorly-named-countries into ones that PyCountry can recognize.

In [ ]:
def sanitize_csv(original, new):
    if new == "nil" or new is None:
        df["Country"] = df["Country"].str.replace(original, "", regex=True)
    else:
        df["Country"] = df["Country"].str.replace(original, new, regex=True)

Prepares the table for use by dropping unneeded columns and renaming an annoying one.


In [ ]:
df.drop(["SNo", "ObservationDate", "Recovered", "Last Update", "Deaths"], axis=1, inplace=True)
df.rename(columns = {"Country/Region": "Country"}, inplace=True)

"Sanitizes" the country names so that PyCountry will recognize them, then collapses them all together.

In [ ]:
for row in bn.itertuples(): sanitize_csv(row[1], row[2])

Groups by and finds the maximum value for each state (the entries are cumulative, so a single group would result in an
absurd amount of cases.)

In [ ]:
df = df.groupby(["Country", "Province/State"], as_index=False, dropna=False).aggregate({"Confirmed":"last"})

Groups the table again by country, finding the sum of all of the states.

In [ ]:
df = df.groupby(["Country"], as_index=False, dropna=False).aggregate({"Confirmed":"sum"})

Uses PyCountry to find the three-letter acronym for each country for use with Folium.

In [ ]:
for row in df["Country"]: df["Country"] = df["Country"].replace(row, pc.countries.search_fuzzy(row)[0].alpha_3)

Takes the log of all values in order to make a more meaningful map. <br>
Without this step, only three countries would be colored anything other than yellow.

In [ ]:
for row in df["Confirmed"]: df["Confirmed"] = df["Confirmed"].replace(row, np.log(row))
df.dropna(inplace=True) # Necessary to remove broken entries created by the above process.

Creates the Folium map.

In [ ]:
outbreak_map = fol.Map(location=[0, 0], zoom_start=0)

fol.Choropleth(
    name = "COVID Cases",
    geo_data = ct, # Polygonal data to draw the country map.
    data = df, # COVID case data
    columns = ["Country", "Confirmed"], # Column to match with the key, count-based column.
    key_on = "feature.id", # Establishes the key of the country JSON.
    fill_color = "YlOrRd", # Color scheme
    fill_opacity = 0.75,
    line_opacity = 0.25,
    nan_fill_opacity = 0,
    legend_name = "Confirmed Cases",
    highlight = True,
).add_to(outbreak_map)

Creates an overlay for the above folium map which displays the confirmed count.

In [ ]:
OverlayData = fol.features.GeoJson(
    mc,
    style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.2, 'weight': 0.2},
    highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1},
    control = False,
    tooltip = fol.features.GeoJsonTooltip(
        fields = ["name", "Confirmed"],
        aliases = ["Country: ", "Confirmed Cases: "],
        style = "background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"
    )
)

outbreak_map.add_child(OverlayData)
outbreak_map.keep_in_front(OverlayData)
fol.LayerControl().add_to(outbreak_map)

## Map of the COVID Outbreak

In [ ]:
outbreak_map

# Project Tasks

## Stage 1 - Using Pandas

### 1.3 - Loading a Dataframe

Loads the dataframe again; the import call is commented out, as it was done earlier.

In [ ]:
# import pandas as pd
CSV_DATA = "https://raw.githubusercontent.com/WBArno/PDA_Project/master/Dat/covid_19_data.csv"

# df = pd.read_csv(CSV_DATA)                  # Loads data from GitHub (CoLab)
df = pd.read_csv("../Dat/covid_19_data.csv")# Loads local data (Local/Git)

### 1.4 - Manipulation with Workflows

#### 1.4a - Datatypes

Displays the datatypes in the dataset.

In [ ]:
print("Data Types:\n", df.dtypes)

#### 1.4b - Top

Displays the top of the dataframe with .head()

In [ ]:
print("Top of Set:\n", df.head())

#### 1.4c - Summary

Creates an automatically-generated statistical summary of the dataframe.

In [ ]:
print("Summary:\n", df.describe())

## Stage 2 - Manipulating and Writing Data

### 2.4 - Adding/Editing Columns

Combines "Province/State" and "Country/Region" into a new column ("Location") in the form "Province, Country".

In [ ]:
p2 = df # Creates a new dataframe for this section
p2["Location"] = p2["Province/State"] + ", " + p2["Country/Region"]
p2

### 2.7a - Creating a Subset

Uses .groupby().agg() to narrow down the table into a more useful form. <br>
Dataset is filtered before other steps to speed up the process.

In [ ]:
# as_index = False so that the next stage is possible.
p2 = p2.groupby(["SNo", "ObservationDate", "Location", "Confirmed", "Recovered", "Deaths"],
                as_index=False).agg({"Recovered":"sum"})
p2

### 2.6 - set_index()

Sets the index to "SNo", which is the original index number for the dataset.

In [ ]:
p2.set_index(["SNo"], inplace=True)
p2

### 2.5 - A Filtering Operation

Filters the dataset so that only entries with a "Recovered" value greater than the mean will be included. <br>
This reduces the dataset by a factor of ten.

In [ ]:
p2 = p2[p2.Recovered > p2.Recovered.mean()]
p2

### 2.7b - to_csv()

Saves the filtered dataframe to "test_output.csv" in the working folder.

In [ ]:
p2.to_csv("test_output.csv")


## Stage 3 - Data Cleansing and Plotting

### Part 3.1 - NaN

#### 3.1.1 - isna()

Finds the number of missing values in the table by column.

In [ ]:
p3 = df
p3.isna().sum()

There are 62,045 missing values in the table; all located under the Province/State column.

#### 3.1.3 - dropna()

Drops all of the rows with null values. <br>
Displays a count of all remaining NaN values to show

In [ ]:
trunc = p3.dropna()
trunc.isna().sum().sum() # Total count of NaN values

#### 3.1.2 - fillna()

Instead of dropping the null values, this replaces them with "Undefined". <br>
Prints a row with a null value before/after the modification to show the change.

In [ ]:
print(p3["Province/State"][35])
p3.fillna("Undefined", inplace=True)
print(p3["Province/State"][35])
p3.isna().sum().sum() # Total count of NaN values.

### Part 3.2 - Plotting

#### 3.2.1 - Histograms

Creates a set of histograms by sorted data:<br>
1) Confirmed Cases:

In [ ]:
# Using the filtered plot from Stage 2 to reduce migraines (caused by the number of data points) slightly.
p2_state_hist = p2["Confirmed"].hist(bins="auto", log=True)

2) Recovered Cases:

In [ ]:
p2_chaos_hist = p2["Recovered"].hist(bins="auto", log=True)

3) Deaths:

In [ ]:
p2_balanced_hist = p2["Deaths"].hist(bins="auto", log=True)

#### 3.2.2 - Plots Within Plots

All three plots, but together now!

In [ ]:
p2_grouped_hist = p2.hist(column=["Confirmed", "Recovered", "Deaths"],
                          bins="auto", layout=(3, 1), figsize=(10, 10), log=True)

#### 3.2.3 - Single-Line Graph

This is an attempt at graphing recovered cases over time. <br>
*It turns out that 20,000 data points don't make for a good line graph.*

In [ ]:
p2["ObservationDate"] = pd.to_datetime(p2["ObservationDate"])
p2.sort_values(by="ObservationDate", ascending=True, inplace=True)
p2.plot(x="ObservationDate", y="Recovered", figsize=(10,5))

#### 3.2.4 - Multi-Line Graph

Now with less data points! <br>
Creates a function to reduce redundant code in this section somewhat.


In [ ]:
def aggravate(tmp_series, target):
    return tmp_series.aggregate({target:"last"}).groupby(["Country/Region"], as_index=False,
                                                         dropna=False).aggregate({target:"sum"})[target]

Creates a temporary *unsorted* dataframe for use with the "aggravate" function.

In [ ]:
tmp_ser = df.groupby(["Country/Region", "Province/State"], as_index=False, dropna=False)

Creates a new dataframe (lg) which is filtered to both make the data more meaningful and reduce the number of datapoints.


In [ ]:
lg = pd.DataFrame()
lg["Country"] = (tmp_ser.aggregate({"Confirmed":"last"}).groupby(["Country/Region"], as_index=False,dropna=False).
                 aggregate({"Confirmed":"sum"}))["Country/Region"]
lg["Confirmed"] = aggravate(tmp_ser, "Confirmed")
lg["Recovered"] = aggravate(tmp_ser, "Recovered")
lg["Deaths"] = aggravate(tmp_ser, "Deaths")

Creates a line graph out of the newly-filtered function using the number of confirmed cases as the x-axis.

In [ ]:
lg.sort_values(by="Confirmed", ascending=True, inplace=True)
lg.plot(
    'Confirmed', # X-Axis index
    title="COVID Case Survival Rates Over Total Cases.",
    ylabel="Cases (Log10)",
    xlabel="Confirmed Cases (Log10)",
    legend=True,
    loglog=True # Sets both axes to a logarithmic scale.
)

mp.pyplot.annotate('|---?---|', (10**7, 10**7))
mp.pyplot.annotate('|----------?----------|', (0.5, 100))

#### 3.2.5 - Bar Graph

Now with beautiful labels! <br>
Saves the artwork to a .png in the working folder.

In [ ]:
lg_bar = lg
lg_bar.set_index("Country")
lg_bar.plot.bar(
    "Country", "Confirmed", # X-Axis, Y-Axis
    title="COVID Cases by Country",
    ylabel="Confirmed Cases (Log10)",
    figsize=(50,5),
    legend=True,
    logy=True
)

mp.pyplot.savefig("not_modern_art.png")

## Stage 4 - Data Wrangling

### Part 4.1 - Hierarchical Indexing

#### Part 4.1.1 - .set_index()

Lorem Ipsum

In [ ]:
# Code goes here.

#### Part 4.1.2 - .unstack()

Lorem Ipsum

In [ ]:
# Code goes here.

#### Part 4.1.3 - Summary Statistics

Lorem Ipsum

In [ ]:
# Code goes here.

#### Part 4.1.4 - .pivot()

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.

#### Part 4.1.5 - .groupby() -> .unstack()

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.

### Part 4.2 - Merge and Concat

#### Part 4.2.1a - Splitting the Dataset

Lorem Ipsum

In [ ]:
# Code goes here.

#### Part 4.2.1b - pd.concat()

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.


#### Part 4.2.2 - pd.merge()


##### Part 4.2.2.1 - The "On" Parameters

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.

##### Part 4.2.2.2 - The "How" Parameter

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.

### Part 4.3 - Pivot Table and CrossTab

#### Part 4.3.1 - .pivot_table()

##### Part 4.3.1.1 - Replicating Hierarchical Indexing

Lorem Ipsum

In [ ]:
# Code goes here.


###### Part 4.3.1.2 - Again, with "margins=True"

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.

###### Part 4.3.1.3 - Yet Again, but with "aggfunc="

Lorem Ipsum

In [ ]:
# Code goes here.

#### Part 4.3.2 - pd.crosstab()

Lorem Ipsum

In [ ]:
# Code goes here.

Explanation goes here.

### Part 4.4 - Screencast Video

*Coming Soon...*
